<a href="https://colab.research.google.com/github/AndreFK/optimus-model/blob/main/tech_topic_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install evaluate
%pip install transformers
%pip install datasets
%pip install accelerate -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached evaluate-0.4.0-py3-none-any.whl (81 kB)
  Using cached datasets-2.12.0-py3-none-any.whl (474 kB)
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
  Using cached xxhash-3.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (212 kB)
  Using cached multiprocess-0.70.14-py310-none-any.whl (134 kB)
  Using cached huggingface_hub-0.15.1-py3-none-any.whl (236 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
  Using cached aiohttp-3.8.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
  Using cached multidict-6.0.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (114 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
  Using cached yarl-1.9.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (268 kB)
  Using cached frozenlist-1.3.3-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.

##Import Pandas and Numpy

In [1]:
import numpy as np
import pandas as pd
import csv

pd.set_option('display.max_colwidth', None)

## TQDM for progress bars in notebooks:

In [2]:
from tqdm.auto import tqdm
tqdm.pandas()

### Load data set with questions from stackoverflow


In [3]:
import nltk, re, string, collections
from nltk.util import ngrams
from nltk.corpus import stopwords
import spacy

stackOver_questions_df = pd.read_csv("/content/drive/MyDrive/askReddit_label.csv")
askReddit_questions_df = pd.read_csv("/content/drive/MyDrive/stackoverflow_label.csv")

In [4]:
stackOver_questions_df.head()

,What's the purpose of life?,0
0,"I've tried to quit smoking, this is my seventh attempt in the last year. I went 2 weeks, started smoking again. What are your stories Reddit on quitting and not being able to quit?",0
1,"For those who have a slave master last name, will you change it? And if so, why?",0
2,How do you think humans will become extinct?,0
3,What is a movie So Disturbing you couldn't be paid to see again?,0
4,Where do banks get their lollipops from?,0


In [5]:
askReddit_questions_df.head()

,"I've written a database generation script in SQL and want to execute it in my Adobe AIR application:\nCreate Table tRole (\n roleID integer Primary Key\n ,roleName varchar(40)\n);\nCreate Table tFile (\n fileID integer Primary Key\n ,fileName varchar(50)\n ,fileDescription varchar(500)\n ,thumbnailID integer\n ,fileFormatID integer\n ,categoryID integer\n ,isFavorite boolean\n ,dateAdded date\n ,globalAccessCount integer\n ,lastAccessTime date\n ,downloadComplete boolean\n ,isNew boolean\n ,isSpotlight boolean\n ,duration varchar(30)\n);\nCreate Table tCategory (\n categoryID integer Primary Key\n ,categoryName varchar(50)\n ,parent_categoryID integer\n);\n...\n\nI execute this in Adobe AIR using the following methods:\npublic static function RunSqlFromFile(fileName:String):void {\n var file:File = File.applicationDirectory.resolvePath(fileName);\n var stream:FileStream = new FileStream();\n stream.open(file, FileMode.READ)\n var strSql:String = stream.readUTFBytes(stream.bytesAvailable);\n NonQuery(strSql);\n}\n\npublic static function NonQuery(strSQL:String):void\n{\n var sqlConnection:SQLConnection = new SQLConnection();\n sqlConnection.open(File.applicationStorageDirectory.resolvePath(DBPATH);\n var sqlStatement:SQLStatement = new SQLStatement();\n sqlStatement.text = strSQL;\n sqlStatement.sqlConnection = sqlConnection;\n try\n {\n sqlStatement.execute();\n }\n catch (error:SQLError)\n {\n Alert.show(error.toString());\n }\n}\n\nNo errors are generated, however only tRole exists. It seems that it only looks at the first query (up to the semicolon- if I remove it, the query fails). Is there a way to call multiple queries in one statement?\n",1
0,Are there any really good tutorials explaining branching and merging with Apache Subversion? \nAll the better if it's specific to TortoiseSVN client.\n,1
1,"Has anyone got experience creating SQL-based ASP.NET site-map providers?\nI've got the default XML file web.sitemap working properly with my Menu and SiteMapPath controls, but I'll need a way for the users of my site to create and modify pages dynamically.\nI need to tie page viewing permissions into the standard ASP.NET membership system as well.\n",1
2,"This is something I've pseudo-solved many times and never quite found a solution. That's stuck with me. The problem is to come up with a way to generate N colors, that are as distinguishable as possible where N is a parameter.\n",1
3,"I have a little game written in C#. It uses a database as back-end. It's \na trading card game, and I wanted to implement the function of the cards as a script.\nWhat I mean is that I essentially have an interface, ICard, which a card class implements (public class Card056 : ICard) and which contains function that are called by the game.\nNow, to make the thing maintainable/moddable, I would like to have the class for each card as source code in the database and essentially compile it on first use. So when I have to add/change a card, I'll just add it to the database and tell my application to refresh, without needing any assembly deployment (especially since we would be talking about 1 assembly per card which means hundreds of assemblies).\nIs that possible? Register a class from a source file and then instantiate it, etc.\nICard Cards[current] = new MyGame.CardLibrary.Card056();\nCards[current].OnEnterPlay(ref currentGameState);\n\nThe language is C#, but extra bonus if it's possible to write the script in any .NET language.\n",1
4,"I am working on a collection of classes used for video playback and recording. I have one main class which acts like the public interface, with methods like play(), stop(), pause(), record() etc... Then I have workhorse classes which do the video decoding and video encoding. \nI just learned about the existence of nested classes in C++, and I'm curious to know what programmers think about using them. I am a little wary and not really sure what the benefits/drawbacks are, but they seem (according to the book I'm re

In [6]:
stackOver_questions_df.columns = ["text", "label"]
askReddit_questions_df.columns = ["text", "label"]

stackoverflow_for_fullSet = stackOver_questions_df[["text", "label"]]

askReddit_for_fullSet = askReddit_questions_df[["text", "label"]]

stackoverflow_for_fullSet.replace(r'\r+|\n+|\t+','', regex=True, inplace=True)

frames = [stackoverflow_for_fullSet, askReddit_for_fullSet]

fullSet = pd.concat(frames)

print("SO: ", stackoverflow_for_fullSet.shape[0])
print("REDDIT: ", askReddit_for_fullSet.shape[0])
print("SO + REDDIT: ", (stackoverflow_for_fullSet.shape[0] + askReddit_for_fullSet.shape[0]))
print("FullSet: ", fullSet.shape[0])


fullSet.head()

SO:  19999
REDDIT:  19999
SO + REDDIT:  39998
FullSet:  39998


,text,label
0,"I've tried to quit smoking, this is my seventh attempt in the last year. I went 2 weeks, started smoking again. What are your stories Reddit on quitting and not being able to quit?",0
1,"For those who have a slave master last name, will you change it? And if so, why?",0
2,How do you think humans will become extinct?,0
3,What is a movie So Disturbing you couldn't be paid to see again?,0
4,Where do banks get their lollipops from?,0


In [7]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding



In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [9]:
from datasets import Dataset

to_ds = Dataset.from_pandas(fullSet)
to_ds = to_ds.remove_columns(['__index_level_0__'])

train_ds = to_ds.train_test_split(test_size=0.2, shuffle=True)
train_ds.column_names
processed_df = train_ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/31998 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [10]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

processed_df.column_names

{'train': ['text', 'label', 'input_ids', 'attention_mask'],
 'test': ['text', 'label', 'input_ids', 'attention_mask']}

In [11]:
import evaluate

accuracy = evaluate.load("accuracy")

In [12]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)


id2label = {0: "NON TECH", 1: "TECH"}
label2id = {"NON TECH": 0, "TECH": 1}

In [18]:
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import create_optimizer, AdamWeightDecay
import tensorflow as tf


num_epochs = 4

batches_per_epoch = len(processed_df)
total_train_steps = int(batches_per_epoch * num_epochs)

In [19]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [20]:
from datasets import Dataset


tf_train_set = model.prepare_tf_dataset(
    processed_df["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    processed_df["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)



In [21]:
import tensorflow as tf

model.compile(optimizer=AdamWeightDecay(
    learning_rate=3e-5
))

In [22]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

In [23]:
callbacks = [metric_callback]

In [24]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=num_epochs, callbacks=callbacks)

Epoch 1/4
   6/1999 [..............................] - ETA: 21:17 - loss: 0.6266

1999/1999 [==============================] - 1585s 787ms/step - loss: 0.0271 - val_loss: 0.0220 - accuracy: 0.9938
Epoch 2/4
1999/1999 [==============================] - 1571s 786ms/step - loss: 0.0079 - val_loss: 0.0478 - accuracy: 0.9904
Epoch 3/4
1999/1999 [==============================] - 1595s 798ms/step - loss: 0.0051 - val_loss: 0.0508 - accuracy: 0.9858
Epoch 4/4
1999/1999 [==============================] - 1604s 802ms/step - loss: 0.0035 - val_loss: 0.0281 - accuracy: 0.9932


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
model.summary()
#model.save_weights('/my_checkpoint')

Model: "tf_distil_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_39 (Dropout)        multiple                  0         
                                                                 
Total params: 66,955,010
Trainable params: 66,955,010
Non-trainable params: 0
_________________________________________________________________


In [26]:
!mkdir -p optimus
model.save_pretrained('drive/MyDrive/optimus/tech-topic')

In [30]:
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

model = TFAutoModelForSequenceClassification.from_pretrained("drive/MyDrive/optimus/tech-topic")

Some layers from the model checkpoint at drive/MyDrive/optimus/tech-topic were not used when initializing TFDistilBertForSequenceClassification: ['dropout_39']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at drive/MyDrive/optimus/tech-topic and are newly initialized: ['dropout_119']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
text = "can you transpile this code"

inputs = tokenizer(text, return_tensors="tf")
logits = model(**inputs).logits

predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])

model.config.id2label[predicted_class_id]

'TECH'

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1
